<a href="https://colab.research.google.com/github/aliakbarbadri/persian-poetry-creator/blob/master/char-gru.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import time

# Load the data

In [0]:
url = "https://raw.githubusercontent.com/aliakbarbadri/persian-poetry-creator/master/shahname2.txt"
filepath = keras.utils.get_file("shahname2.txt", url) 
corpus = open(filepath, 'rb').read().decode(encoding='utf-8')
corpus = corpus.replace("\t"," \t ").replace("\n", " \n ")

In [6]:
vocab = sorted(set(corpus))
print(len(vocab), "chars")

48 chars


In [7]:
vocab

['\t',
 '\n',
 ' ',
 '(',
 ')',
 '«',
 '»',
 '،',
 '؟',
 'ء',
 'آ',
 'أ',
 'ؤ',
 'ئ',
 'ا',
 'ب',
 'ت',
 'ث',
 'ج',
 'ح',
 'خ',
 'د',
 'ذ',
 'ر',
 'ز',
 'س',
 'ش',
 'ص',
 'ض',
 'ط',
 'ظ',
 'ع',
 'غ',
 'ف',
 'ق',
 'ل',
 'م',
 'ن',
 'ه',
 'و',
 'ٔ',
 'پ',
 'چ',
 'ژ',
 'ک',
 'گ',
 'ی',
 '\u200c']

In [0]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in corpus])

In [0]:
seq_length = 1000
examples_per_epoch = len(corpus)//(seq_length+1)
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [11]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((64, 1000), (64, 1000)), types: (tf.int64, tf.int64)>

# Model (GRU)

In [0]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

In [28]:
model = keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[BATCH_SIZE, None]))
model.add(tf.keras.layers.GRU(rnn_units,return_sequences=True,stateful=True,
                              recurrent_initializer='glorot_uniform',
                              recurrent_regularizer=keras.regularizers.l2(0.01)
                              ))
model.add(tf.keras.layers.Dense(vocab_size))
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (64, None, 256)           12288     
_________________________________________________________________
gru_8 (GRU)                  (64, None, 1024)          3938304   
_________________________________________________________________
gru_9 (GRU)                  (64, None, 1024)          6297600   
_________________________________________________________________
dense_4 (Dense)              (64, None, 48)            49200     
Total params: 10,297,392
Trainable params: 10,297,392
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer='adam', loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

In [30]:
history = model.fit(dataset, epochs=50)

Epoch 1/50
42/42 [==============================] - 110s 3s/step - loss: 10.9812 - accuracy: 0.2489
Epoch 2/50
42/42 [==============================] - 110s 3s/step - loss: 2.4215 - accuracy: 0.3313
Epoch 3/50
42/42 [==============================] - 111s 3s/step - loss: 2.2212 - accuracy: 0.3547
Epoch 4/50
42/42 [==============================] - 111s 3s/step - loss: 2.1451 - accuracy: 0.3746
Epoch 5/50
42/42 [==============================] - 110s 3s/step - loss: 2.0956 - accuracy: 0.3931
Epoch 6/50
42/42 [==============================] - 110s 3s/step - loss: 2.0268 - accuracy: 0.4128
Epoch 7/50
42/42 [==============================] - 111s 3s/step - loss: 1.9699 - accuracy: 0.4326
Epoch 8/50
42/42 [==============================] - 111s 3s/step - loss: 1.9316 - accuracy: 0.4451
Epoch 9/50
42/42 [==============================] - 111s 3s/step - loss: 1.8570 - accuracy: 0.4653
Epoch 10/50
42/42 [==============================] - 111s 3s/step - loss: 1.8233 - accuracy: 0.4758
Epoch 11

In [31]:
main_model = keras.Sequential()
main_model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[1, None]))
main_model.add(tf.keras.layers.GRU(rnn_units,return_sequences=True,stateful=True,
                              recurrent_initializer='glorot_uniform',
                              recurrent_regularizer=keras.regularizers.l2(0.01)
                              ))
main_model.add(tf.keras.layers.Dense(vocab_size))

main_model.set_weights(model.get_weights())
model = main_model
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (1, None, 256)            12288     
_________________________________________________________________
gru_10 (GRU)                 (1, None, 1024)           3938304   
_________________________________________________________________
gru_11 (GRU)                 (1, None, 1024)           6297600   
_________________________________________________________________
dense_5 (Dense)              (1, None, 48)             49200     
Total params: 10,297,392
Trainable params: 10,297,392
Non-trainable params: 0
_________________________________________________________________


In [0]:
def generate_text(model, start_string, temperature = 1.0):
  num_generate = 1000
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)
  text_generated = []
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      predictions = tf.squeeze(predictions, 0)
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
      input_eval = tf.expand_dims([predicted_id], 0)
      text_generated.append(idx2char[predicted_id])
  return (start_string + ''.join(text_generated))

In [33]:
print(generate_text(model, start_string=u"که ایران چوباغیست خرم بهار", temperature=1))

که ایران چوباغیست خرم بهار 	 کسی را نزد دردن دمید رزم 
 چو فرم‌آذر او رای و دور گرفت 	 سیه سپخمه برین روز بگذوش 
 بدو گفت رستم به را خواندند 	 میان به افگند انددازه راه 	 بخور شاهان بیامد بدرد 
 کمان نامور زار دیدار شد 	 چوگفتی ناگزند ار شغوری 
 فروبر برامافراختندی 	 پرستنده ایران گروهان برد 
 جگر بر خسته از دولتر رسید 	 درم درم بند هر نهاد 
 سزد کو سسست این چون گلاب 	 نسودیم مردان مهره کرد 
 ز شاهان تیرز فروزد بخوی 	 چو اندر تنگ دانا بران 
 هوا ای بپرسیدش اندر چنین 	 گزین کرد خرگیر و لب 
 دگر رنگ رنگاه باید قباد 	 ز دشمن پویلان تا کنده‌ایم 
 بدو گفت رستم بدید و درد و کژ گرد 	 سه جان شبین سوی و انبرید و بنم 
 چو خورشید خاقان خردمند خواستار 	 ببخشید و کاندر کن مهان 
 گناه ترکان که پور شد مریز و گلاغ 	 بزرگان تا به آیین ناسپ افگند 
 بدو گفت رستمست از کند 	 بر در تنان بخسرود شارستان 
 سوی را همی‌داشت از روی شست 	 نبوندی ز پس آتش هوا رویک بود 
 برآمدم تا سرجام نزدیک 	 دل باید و رخ چورگ تابند گل 
 دگر راز کان نام و خیره فریدون 	 همان ز لشکر ز تیغ و تن 	 نیایدش برین همی جای تو هست 
 دلش چاره

# Model (LSTM)

In [39]:
model = keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[BATCH_SIZE, None]))
model.add(tf.keras.layers.LSTM(rnn_units,return_sequences=True,stateful=True,
                              recurrent_initializer='glorot_uniform',
                              recurrent_regularizer=keras.regularizers.l2(0.01)
                              ))
model.add(tf.keras.layers.Dense(vocab_size))
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (64, None, 256)           12288     
_________________________________________________________________
lstm_6 (LSTM)                (64, None, 1024)          5246976   
_________________________________________________________________
dense_9 (Dense)              (64, None, 48)            49200     
Total params: 5,308,464
Trainable params: 5,308,464
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer='adam', loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

In [41]:
history = model.fit(dataset, epochs=20)

Epoch 1/20
42/42 [==============================] - 62s 1s/step - loss: 8.3038 - accuracy: 0.2201
Epoch 2/20
42/42 [==============================] - 62s 1s/step - loss: 2.9113 - accuracy: 0.3026
Epoch 3/20
42/42 [==============================] - 62s 1s/step - loss: 2.4002 - accuracy: 0.3359
Epoch 4/20
42/42 [==============================] - 62s 1s/step - loss: 2.2880 - accuracy: 0.3443
Epoch 5/20
42/42 [==============================] - 62s 1s/step - loss: 2.2354 - accuracy: 0.3524
Epoch 6/20
42/42 [==============================] - 62s 1s/step - loss: 2.1844 - accuracy: 0.3634
Epoch 7/20
42/42 [==============================] - 62s 1s/step - loss: 2.1558 - accuracy: 0.3690
Epoch 8/20
42/42 [==============================] - 63s 1s/step - loss: 2.1112 - accuracy: 0.3810
Epoch 9/20
42/42 [==============================] - 63s 1s/step - loss: 2.0484 - accuracy: 0.3982
Epoch 10/20
42/42 [==============================] - 62s 1s/step - loss: 1.9929 - accuracy: 0.4127
Epoch 11/20
42/42 [

In [44]:
main_model = keras.Sequential()
main_model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[1, None]))
main_model.add(tf.keras.layers.LSTM(rnn_units,return_sequences=True,stateful=True,
                              recurrent_initializer='glorot_uniform',
                              recurrent_regularizer=keras.regularizers.l2(0.01)
                              ))
main_model.add(tf.keras.layers.Dense(vocab_size))

main_model.set_weights(model.get_weights())
main_model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (1, None, 256)            12288     
_________________________________________________________________
lstm_8 (LSTM)                (1, None, 1024)           5246976   
_________________________________________________________________
dense_11 (Dense)             (1, None, 48)             49200     
Total params: 5,308,464
Trainable params: 5,308,464
Non-trainable params: 0
_________________________________________________________________


In [48]:
print(generate_text(main_model, start_string=u"که ایران چوباغیست خرم بهار", temperature=1.5))

که ایران چوباغیست خرم بهار 
 یکایچ آزود ناش پیایمشن 	 به روششید بستندگنع زره غم 
 گمیل وزورگ‌ان به آراغ کشاد 
 جهای دشامر آفرزگان آتیخ 	 پردده توراب دازشتر دیده‌شگردگانه 
 بوم کسگه اندرد بنشایش 	 شنش نکسند منس رطاو گویرهک کرترگ 	 چوزوآتسپ باپوس وتخت یکی 	 به جزشم نگراوی فنعتابود 	 همه خمی نشست تازیزه دست بازخیم 
 ترابری فرش از راستیچورهٔ 	 درنگند درست کآوکاه سرید 
 نیامد بهفالستن شمشود کرد 	 برآیدش شاهی دله ای خسزون 
 من بپیرویان وسبد نستیم گفت 
 وفغن برمفدند بستی کمانی بگ 	 هرک شک جایدیودن بهفند هم 	 بپریاوننید آفران گرز شهنخت 
 چنین یزهدونش بوپیچ و رسید ج 	 هر کاوشگد چو شهر فراستورفزای 	 که آهیبای دل او دیقان طوس 
 امراده دتر بگذیر جامگهاد 
 اسپی اندش هزارگاند اعن فوند ودشت 
 خردم اورد هفتا کزاد ویبسگاه 	 نینم آگنک میدنش سودفوی 
 جهانش آتب ویسراگانگرد دوی 	 خواسان یلچ ازو رختش جای بخردن 	 هش جونیخضت باشد نشاراسر 
 فر آقبجلذر سپارش آنشکچ 
 خوبرد چوند بساعنا بفرم کنست 
 بر بیشهن بلشکرا نه ایموراند 
 چوان خدا دیگر سپهرسفزوند 	 سوی هر زدن فتی دابید گرف 
 سبند ر سه منسی‌گر پسرد دبازین 
 و

In [0]:
model.save("char_lstm.h5")